In [1]:
import numpy
import random

In [2]:
def copying_data(T=30, n_data=300*64, n_sequence=10, batch_size=64, make_rand=False, shuffle=False, drop_last=False):
    seq = numpy.random.randint(1, high=9, size=(n_data, n_sequence))
    zeros1 = numpy.zeros((n_data, T-1))
    zeros2 = numpy.zeros((n_data, T))
    marker = 9 * numpy.ones((n_data, 1))
    zeros3 = numpy.zeros((n_data, n_sequence))

    x = numpy.concatenate((seq, zeros1, marker, zeros3), axis=1).astype('int32')
    y = numpy.concatenate((zeros3, zeros2, seq), axis=1).astype('int64')

    x = numpy.expand_dims(x, axis=-1)
    y = numpy.expand_dims(y, axis=-1)
    print(x.shape)
    print(y.shape)
    return x, y

In [3]:
random.seed(1)

train_x, train_y = copying_data(T=50)
test_x, test_y   = copying_data(T=200)

(19200, 70, 1)
(19200, 70, 1)
(19200, 220, 1)
(19200, 220, 1)


# Model comparison

In [4]:
import tensorflow as tf
from tensorflow.keras.layers import LSTMCell
from tfRIM import RIMCell

## LSTM model

In [5]:
tf.random.set_seed(1234)
lstmcell = LSTMCell(600)

In [6]:
inputs = tf.keras.Input([None, 1])
rnnout = tf.keras.layers.RNN(lstmcell, return_sequences=True)(inputs)
seqout = tf.keras.layers.Dense(10, activation='softmax')(rnnout)
output = seqout[:, -70:]
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
cost = tf.keras.losses.SparseCategoricalCrossentropy()

model_lstm = tf.keras.Model(inputs, output)
model_lstm.compile(optimizer=opt, loss=cost)

In [7]:
tf.random.set_seed(1234)
model_lstm.fit(train_x, train_y, batch_size=128, epochs=10)

Epoch 1/10
150/150 [==============================] - 13s 67ms/step - loss: 0.6428
Epoch 2/10
150/150 [==============================] - 10s 64ms/step - loss: 0.3683
Epoch 3/10
150/150 [==============================] - 9s 61ms/step - loss: 0.3279
Epoch 4/10
150/150 [==============================] - 9s 60ms/step - loss: 0.2978
Epoch 5/10
150/150 [==============================] - 9s 60ms/step - loss: 0.2938
Epoch 6/10
150/150 [==============================] - 9s 59ms/step - loss: 0.2923
Epoch 7/10
150/150 [==============================] - 9s 59ms/step - loss: 0.2917
Epoch 8/10
150/150 [==============================] - 9s 60ms/step - loss: 0.2912
Epoch 9/10
150/150 [==============================] - 9s 61ms/step - loss: 0.2910
Epoch 10/10
150/150 [==============================] - 9s 61ms/step - loss: 0.2892


## RIMs model

In [8]:
tf.random.set_seed(1234)
rimcell = RIMCell(units=100, nRIM=6, k=4,
                num_input_heads=1, input_key_size=32, input_value_size=32, input_query_size=32, input_keep_prob=0.9,
                num_comm_heads=4, comm_key_size=32, comm_value_size=32, comm_query_size=32, comm_keep_prob=0.9)

In [9]:
inputs = tf.keras.Input([None, 1])
rnnout = tf.keras.layers.RNN(rimcell, return_sequences=True)(inputs)
seqout = tf.keras.layers.Dense(10, activation='softmax')(rnnout)
output = seqout[:, -70:]
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
cost = tf.keras.losses.SparseCategoricalCrossentropy()

model_rims = tf.keras.Model(inputs, output)
model_rims.compile(optimizer=opt, loss=cost)

In [10]:
tf.random.set_seed(1234)
model_rims.fit(train_x, train_y, batch_size=128, epochs=10) 

Epoch 1/10
150/150 [==============================] - 35s 216ms/step - loss: 0.8074
Epoch 2/10
150/150 [==============================] - 33s 219ms/step - loss: 0.3484
Epoch 3/10
150/150 [==============================] - 32s 216ms/step - loss: 0.3002
Epoch 4/10
150/150 [==============================] - 32s 215ms/step - loss: 0.2973
Epoch 5/10
150/150 [==============================] - 32s 214ms/step - loss: 0.3309
Epoch 6/10
150/150 [==============================] - 33s 219ms/step - loss: 0.3009
Epoch 7/10
150/150 [==============================] - 32s 215ms/step - loss: 0.2969
Epoch 8/10
150/150 [==============================] - 33s 219ms/step - loss: 0.2948
Epoch 9/10
150/150 [==============================] - 32s 216ms/step - loss: 0.2985
Epoch 10/10
150/150 [==============================] - 32s 215ms/step - loss: 0.2981


## Evaluate

#### In sample prediction

In [11]:
ypred1_lstm = model_lstm.predict(train_x)
ypred1_rims = model_rims.predict(train_x)

cost = tf.keras.losses.SparseCategoricalCrossentropy()

ValueError: ignored

In [12]:
ce_train_lstm = cost(train_y[:,-70:], ypred1_lstm)
ce_train_rims = cost(train_y[:,-70:], ypred1_rims)

ac_train_lstm = (numpy.argmax(ypred1_lstm, axis=-1) == train_y[:,-70:,0]).mean()
ac_train_rims = (numpy.argmax(ypred1_rims, axis=-1) == train_y[:,-70:,0]).mean()

In [13]:
ce_train_lstm, ce_train_rims

(<tf.Tensor: shape=(), dtype=float32, numpy=0.28548497>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.2966531>)

In [14]:
ac_train_lstm, ac_train_rims

(0.8844575892857143, 0.8781629464285714)

#### Out of sample prediction

In [15]:
ypred2_lstm = model_lstm.predict(test_x)
ypred2_rims = model_rims.predict(test_x)

cost = tf.keras.losses.SparseCategoricalCrossentropy()

ce_test_lstm = cost(test_y[:,-70:], ypred2_lstm)
ce_test_rims = cost(test_y[:,-70:], ypred2_rims)

ac_test_lstm = (numpy.argmax(ypred2_lstm, axis=-1) == test_y[:,-70:,0]).mean()
ac_test_rims = (numpy.argmax(ypred2_rims, axis=-1) == test_y[:,-70:,0]).mean()

In [16]:
ce_test_lstm, ce_test_rims

(<tf.Tensor: shape=(), dtype=float32, numpy=10.653892>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.296768>)

In [17]:
ac_test_lstm, ac_test_rims

(0.03215327380952381, 0.8752113095238095)